# Import the dependent libraries

In [ ]:
"""import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv #Just in case we need an API key.
import requests
import json
import numpy as np
%matplotlib inline
from datetime import date
from datetime import timedelta
#Import Various Libraries, including Tweepy, a Python library for the Twitter API.
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from textblob import TextBlob 
#%matplotlib inline
import tweepy

load_dotenv()
pd.options.display.float_format = '{:.2f}'.format"""

In [ ]:
#Pass API Keys to Twitter API and build Tweepy API handler object.
consumer_key = os.getenv('ALT_TWITTER_API_KEY')
consumer_secret = os.getenv('ALT_TWITTER_SECRET_KEY')
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [ ]:
#Creating a DataFrame for tweet search results data structure.
dict_tweet_structure = {"twitter_user":"","category":[],"time":"","sentiment":"","text":"","tweet_id":"",
                        "tweet_source":"","quote_count":0,"reply_count":0,"retweet_count":0,"fav_count":0,
                        "Polarity Rating": "", "Popularity Rating":""}

In [ ]:
#Initializing primary DataFrame of tweets. DON'T RUN! Or you will lose your data. :)
df_tweets_found = pd.DataFrame(dict_tweet_structure)

In [ ]:
#Define Search Term Library
dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin',],
                     'generic':['cryptocurrency','blockchain'],
                     'litecoin':['litecoin','ltc','#litecoin', '#ltc'],
                     'ethereum':['ethereum','eth','#ethereum','#eth']   }
#Define Twitter User List
list_twitterers = ['joerogan','elonmusk','officialmcafee','vitalikbuterin']

In [ ]:
#Function for creating search query string for tweepy.Cursor(api.search). Max query length is 128 for sandbox env.
#In it's current form it supports a bunch of keywords joined by OR, grouped by (), AND from a single tweeter.
def query_creator(list_keywords, twitter_user = "!", mood = "!"):
    query = "("
    for keyword in (list_keywords):
        query += keyword
        if keyword != list_keywords[-1]:
            query += " OR "
        else:
            query += ")"
    if twitter_user != "!":
        query += " from:" + twitter_user
    return query

In [ ]:
#This is the core function for tweet searching.
def tweet_search_full(string_query, date_from = "200603210000", date_to = "201801010000", number_tweets = 500):
    for tweets in tweepy.Cursor(api.search_full_archive, query=string_query, environment_name='CryptoSentimentQueryProd',
                                fromDate=date_from, toDate=date_to).items():
        #Primary environment name = CryptoSentimentFullArc
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), "", contents, tweets.id, tweets.source, tweets.quote_count,
               tweets.reply_count, tweets.retweet_count, tweets.favorite_count]

In [ ]:
#This function searches the last month, but uses the same arguments and formatting as full archive; useful for testing without using up queries.
def tweet_search_month(string_query, date_from = "200603210000", date_to = "202101170000", number_tweets = 500):
    for tweets in tweepy.Cursor(api.search_30_day, query=string_query, environment_name='CryptoSentimentQueryTest',
                                fromDate=date_from, toDate=date_to,).items():
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), str(tweets.coordinates), contents, tweets.id, tweets.source, tweets.quote_count,
               tweets.reply_count, tweets.retweet_count, tweets.favorite_count]
        

In [ ]:
def tweet_search_free(string_query, number_tweets = 100):
    for tweets in tweepy.Cursor(api.search, q=string_query).items(number_tweets):
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        polarity = sentiment_reader(contents)
        
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), str(tweets.coordinates), contents, tweets.id, tweets.source,"x",
               "x", tweets.retweet_count, tweets.favorite_count, polarity, (polarity * int(tweets.retweet_count))]

In [ ]:
#Function for appending search results to the tweet df.
def tweet_dataframe_append(rows, target_dataframe):
    for row in rows:
        series_result = pd.Series(row, index=target_dataframe.columns)
        target_dataframe = target_dataframe.append(series_result, ignore_index=True)
        return target_dataframe

In [ ]:
#Function for generating a list of categories from the presence of keywords in text.
def category_key(text_block,dict_keywords):
    category_list = []
    output = " "
    contents = text_block.lower()
    for key in dict_keywords:
        for keyword in dict_keywords[key]:
            if keyword in contents:
                category_list.append(key)
    #return output.join(category_list)
    return category_list

In [ ]:
#Function for generating sentiment values using textblob library.
def sentiment_reader(text_block):
    sentiment_text = TextBlob(text_block)
    return sentiment_text.polarity

In [424]:
#Function for fixing duplicate values in the category column.
def no_dupes(string):
    string =  str(string)
    if string[-1] != " ":
        string = string + " "
    wordlist = []
    output = ""
    word = ""
    for letter in string:
        if letter != " ":
            word = word + letter
        elif word not in wordlist:
            wordlist.append(word)
            output = output + word + " "
            word = ""
        else:
            word = ""
    return output

In [ ]:
#Testing Query Creator
query = query_creator(dict_search_terms['generic']+dict_search_terms['bitcoin']+dict_search_terms['litecoin'], 'satoshilite')
print(query)
print(len(query))
#Query length is limited to 128 characters, max tweets per query is limited to 100.

In [ ]:
#Testing primary search and DataFrame append.
search_results = tweet_search_full(query)

for tweets in search_results:
    print (tweets)
    series_result = pd.Series(tweets, index=df_tweets_found.columns)
    df_tweets_found = df_tweets_found.append(series_result, ignore_index=True)

In [425]:
df_tweets_found.to_csv('../data/raw_data/raw_tweets_01.csv',index=False)

NameError: name 'df_tweets_found' is not defined

In [ ]:
df_cleaning = pd.read_csv('../data/raw_data/raw_tweets_01_filter.csv')
#df_cleaning.drop_duplicates(inplace=True)
#df_cleaning.xs('text', axis=1)
df_cleaning.loc[13]['text']

In [ ]:
#Run several times to eliminate usernames containing crypto.
df_cleaning = df_cleaning[~df_cleaning.text.str.contains("_crypto")]

In [ ]:
df_cleaning.to_csv('../data/raw_data/raw_tweets_01_filter.csv',index=False)
df_tweets_found
#df_cleaning.head(100)

In [ ]:
##trying to test sentiment analysis 
df_tweets_found = pd.read_csv('../data/raw_data/raw_tweets_01_filter.csv')

#pull the tweet text as a list from the data frame
tweet_text_list = df_tweets_found.text.tolist()


#create objects for sentiment

sentiment_text = [TextBlob(tweet) for tweet in tweet_text_list]

sentiment_text[0].polarity , sentiment_text[0]

#create sentiment list

sentiment_list = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_text]
sentiment_list[0]
#remove the tweet from the list of lists
for x in sentiment_list:
    del x[1]
# convert the list of lists into a DF column
df_polarity = pd.DataFrame(sentiment_list, columns = ['Polarity'])
#convert back to a single list
polarity_list = df_polarity["Polarity"].tolist()
#append to OG data frame

df_tweets_found['Polarity Rating'] = polarity_list

df_tweets_found["Popularity Rating"]= df_tweets_found["fav_count"] * df_tweets_found["Polarity Rating"]
df_tweets_found.to_csv('../data/raw_data/raw_tweets_01_filter_polarity.csv',index=False)

In [426]:
#Using recent search to generate recent tweets from anyone that include our keywords.
df_tweets_live = pd.DataFrame(dict_tweet_structure)
query = query_creator(dict_search_terms['generic']+dict_search_terms['bitcoin']+dict_search_terms['litecoin']+dict_search_terms['ethereum'])
print(query)
live_results = tweet_search_free(query, 1000)
for tweets in live_results:
    #print (tweets)
    df_tweets_live = tweet_dataframe_append(live_results, df_tweets_live)
    

NameError: name 'dict_tweet_structure' is not defined

In [ ]:
import hvplot.pandas
#df_tweets_live.hvplot()
df_tweets_live.head()

In [ ]:
polarity_test = df_tweets_live#.filter(['time','Popularity Rating'] ,axis=1)#.hvplot(x='time',rot=90)
polarity_test

In [ ]:
polarity_test['time'] = pd.to_datetime(polarity_test['time'],format= '%M:%S',infer_datetime_format=True ).dt.time
rolling_poll = polarity_test['Polarity Rating'].rolling(window=20).mean()
end = pd.to_timedelta([str(polarity_test['time'].iloc[0])])
start = pd.to_timedelta([str(polarity_test['time'].iloc[-1])])
recent_tweets_duration = str((end-start)[0])
roll = rolling_poll.hvplot(x='time',y='Polarity Rating',rot=90, hover_cols = ['twitter_user','text'])
pol = polarity_test.hvplot(x='time',y='Polarity Rating',rot=90, hover_cols = ['twitter_user','text'],xticks=6,xlabel=f"Duration length = {recent_tweets_duration}")
#print(polarity_test['Polarity Rating'].mean())
#print((polarity_test['time'].iloc[-1]))
#print((polarity_test['time'].iloc[0]))
#polarity_test.mean()
pol



In [ ]:
end = pd.to_timedelta([str(polarity_test['time'].iloc[0])])
start = pd.to_timedelta([str(polarity_test['time'].iloc[-1])])
print ((end-start)[0])
recent_tweets_duration = str((end-start)[0])

In [ ]:
df_graphing = pd.read_csv('../data/raw_data/raw_tweets_01_filter_polarity.csv')

In [ ]:
df_graphing = df_graphing.set_index('time')
df_graphing.sort_index(inplace=True)
df_graphing

In [ ]:
df_graphing.drop('sentiment',axis = 1, inplace=True)

In [ ]:
df_graphing = df_graphing.reset_index()
#pd.unique(df_graphing['category'])
#df_bitcoin = df_graphing.groupby('category')
#df_bitcoin.head()
df_graphing['category'] = df_graphing.category.apply(no_dupes)
#df_graphing.to_csv("../data/raw_data/raw_tweets_cleaning.csv",index=False)
df_graphing['time'] = pd.to_datetime(df_graphing['time'],format= '%b:%d:%Y',infer_datetime_format=True ).dt.date #
#df_graphing
df_graphing = df_graphing.set_index('time')
df_graphing.sort_index(inplace=True)
df_bitcoin = df_graphing.loc[df_graphing.category.str.contains('bitcoin')]
df_ethereum = df_graphing.loc[df_graphing.category.str.contains('ethereum')]
df_litecoin = df_graphing.loc[df_graphing.category.str.contains('litecoin')]
df_generic = df_graphing.loc[df_graphing.category.str.contains('generic')]
df_bitcoin_rolling = df_bitcoin['Polarity Rating'].rolling(window=50).mean()
df_ethereum_rolling = df_ethereum['Polarity Rating'].rolling(window=50).mean()
df_litecoin_rolling = df_litecoin['Polarity Rating'].rolling(window=50).mean()
df_generic_rolling = df_generic['Polarity Rating'].rolling(window=50).mean()
#df_bitcoin_rolling#.hvplot(rot=90)


In [ ]:
df_graphing.to_csv("../data/raw_data/cleaner_tweets.csv")

In [427]:
bitcoin_roller * ethereum_roller * generic_roller * litecoin_roller

NameError: name 'bitcoin_roller' is not defined

In [ ]:
df_graphing.hvplot(groupby='category')

In [ ]:
graph_bitcoin = df_bitcoin.hvplot(x='time',y='Popularity Rating',rot=90, hover_cols = ['twitter_user','text'])
graph_ethereum = df_ethereum.hvplot(x='time',y='Popularity Rating',rot=90, hover_cols = ['twitter_user','text'])
graph_litecoin = df_litecoin.hvplot(x='time',y='Popularity Rating',rot=90, hover_cols = ['twitter_user','text'])
graph_generic = df_generic.hvplot(x='time',y='Popularity Rating',rot=90, hover_cols = ['twitter_user','text'])
graph_etheruem + graph_bitcoin

In [ ]:
#This is the user_timeline method, which seems to return around 500~1000 tweets tops.
ticker = 0
for tweet in tweepy.Cursor(api.user_timeline, id='elonmusk',trim_user=True, max_id=2237531699681981416, count =10000, exclude_replies = True).items():
    #if 'bitcoin' in tweet.text:
    ticker += 1
    print(tweet.text + " " + str(tweet.created_at)+" "+str(tweet.id)+"\n")
print(ticker)

In [ ]:
for tweet in tweepy.Cursor(api.user_timeline, id="elonmusk",tweet_mode="extended").items(100):
    #if 'Python' in tweet.full_text:
        print(tweet.full_text)
        #print(tweet.user['id_str'])

In [ ]:
#Defunct Old Search Method
def tweet_gather(keywords, user_id):
    #contents = ""
    for tweet in tweepy.Cursor(api.user_timeline, id=user_id).items():
        """if tweet.truncated == True:
            contents = tweet.full_text
        else:
            contents = str(tweet.text).lower()"""
        
        contents = tweet.text.lower()
        #contents = contents.lower() 
        for word in keywords:           
            if word.lower() in contents:
                #print(tweet.text + " " + str(tweet.created_at)+"\n")
                """Here we can append to a pd.DataFrame that collects the twitter handle, keywords used, keyword category
                   for ex:'bitcoin' for hits on 'bitcoin' or 'btc', as well as any sentiment rating we create.
                   Ulitmately we can then export that DataFrame to csv file."""
                yield tweet.text, str(tweet.created_at)

In [ ]:

#This is the basic api.search; it only returns tweets from the last 7 days.
for tweet in tweepy.Cursor(api.search, q='(cryptocurrency OR crypto OR blockchain OR bitcoin OR struggle OR python)').items(1000):
    print(tweet.text + " " + str(tweet.created_at)+" "+str(tweet.place)+"\n")

In [ ]:
#Function for creating search query string for tweepy.Cursor(api.search_full_archive).
def query_creator_full(list_keywords, twitter_user = "!", mood = "!"):
    query = " "
    query = query.join(list_keywords)
    #print(query)
    if twitter_user != "!":
        query += " from:" + twitter_user
        #print(query)
    return query

## Fetch the data from Coinbase API
- TODO
    - Need to show data before dataframe conversion (show our process before creating function)
    - Explain how we got column names
    - 

In [395]:
def fetch_daily_data(symbol, start, end):
    pair_split = symbol.split('/') # Splitting our symbol by the '/' and creating a a list for the new values.
    symbol = pair_split[0] + '-' + pair_split[1] # symbol = BTC-USD #The API request format requires the dash.
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?start={start}&end={end}&granularity=86400'#notice the symbol insert. There are 86400 seconds in a day.
    response = requests.get(url) #getting response from website
    if response.status_code == 200: # check to make sure the response from server is good
        #if response is good then we create a dataframe by reformatting a json load.
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s') # convert to a readable date
       #######

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata_{end}.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

## Call the fetch function with a function focused on our three main cryptos: BTC/USD, ETH/USD, LTC/USD.

In [396]:
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.strftime("%Y-%m-%d")
#start_date = yesterday - timedelta(days = 300)#wont let me get more then 298 days, or so.
start_date = '2015-01-01'
end_date = '2015-09-30'
#start_date = start_date.strftime("%Y-%m-%d")
#end_date = end_date.strftime("%Y-%m-%d")
cryptolist = ['BTC/USD', 'ETH/USD', 'LTC/USD']

#function to pull crypto data from coinbase api passing in crypto symbol pair, start and end date,
def fetch_main_cryptos(crypto):
    fetch_daily_data(crypto, start_date, end_date)

    
#call the function calling our API loop thrgouh crypto list and pull based on start/end date
for crypto in cryptolist:
    fetch_main_cryptos(crypto)


## Created a path to our newly created CSV files

In [397]:
BTC_path, ETH_path, LTC_path = (Path('../Justin_edits/Coinbase_BTCUSD_dailydata.csv'),
                                Path('../Justin_edits/Coinbase_ETHUSD_dailydata.csv'),
                                Path('../Justin_edits/Coinbase_LTCUSD_dailydata.csv'))
BTC_df, ETH_df, LTC_df = (pd.read_csv(BTC_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(ETH_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(LTC_path, index_col='date', infer_datetime_format=False, parse_dates=True))

## 1

In [398]:
#This function is to create our main datasets. Please edit and comment on how we should approach this.
#user, category, date, polarity, popularity, fav_count, volume, 
def clean_data(df):
    df = df.dropna() # immediately drop any null values
    df = df.drop(columns=['unix']).copy() #create deep copy of df with desired columns
    df['volume_change'] = round(df['volume'].pct_change(), 2) #find daily percent change in volume
    df['percent_volatility'] = round(((df['high'] - df['low']) / df['high']) * 100, 2) #Finding the amount of change between the low and high, then comparing it to the high.
    df['daily_change'] = round(df['close'].pct_change(), 5) # daily pct change
    df['daily_avg_price'] = round((df['open'] + df['close']) / 2, 2)
    df['rolling_volatility'] = round(df['percent_volatility'].rolling(30).mean(), 2)
    df['avg_volume_change'] = df['volume_change'].rolling(30).mean()
    df['rolling_volume'] = round(df['volume'].rolling(3).mean(), 2)
    df.drop(df.head(2).index, inplace=True) # drop the unfinished and upcoming day, inclusive of NA data
    df.drop(columns=['low', 'close', 'open'], inplace=True)
    df.sort_index(inplace=True)
    return pd.DataFrame(df)
def clean_new_data(df):
    return df.dropna()
#these are the base data sets so far
BTC_df = clean_data(BTC_df).dropna()
ETH_df = clean_data(ETH_df)
LTC_df = clean_data(LTC_df)

## Current Dataframes

## Creating new data frames to compare statistical columns

In [401]:
#joined columns into new dataframe and renamed
BTC_volume = BTC_df['volume']
ETH_volume = ETH_df['volume']
LTC_volume = LTC_df['volume']
volume_df = clean_new_data(pd.concat([BTC_volume, ETH_volume, LTC_volume], axis=1))
volume_df.columns = ['BTC_volume', 'ETH_volume', 'LTC_volume']

#joined columns into new dataframe and renamed
BTC_rolling_volume = BTC_df['rolling_volume']
ETH_rolling_volume = ETH_df['rolling_volume']
LTC_rolling_volume = LTC_df['rolling_volume']
rolling_volume_change_df = clean_new_data(pd.concat([BTC_rolling_volume, ETH_rolling_volume, LTC_rolling_volume], axis=1))
rolling_volume_change_df.columns = ['BTC_rolling_volume', 'ETH_rolling_volume', 'LTC_rolling_volume']


#joined columns into new dataframe and renamed
BTC_volume_change = BTC_df['volume_change']
ETH_volume_change = ETH_df['volume_change']
LTC_volume_change = LTC_df['volume_change']
volume_change_df = clean_new_data(pd.concat([BTC_volume_change, ETH_volume_change, LTC_volume_change], axis=1))
volume_change_df.columns = ['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change']

#joined columns into new dataframe and renamed
BTC_volatilityr = BTC_df['rolling_volatility']
ETH_volatilityr = ETH_df['rolling_volatility']
LTC_volatilityr = LTC_df['rolling_volatility']
volatilityr_df = clean_new_data(pd.concat([BTC_volatilityr, ETH_volatilityr, LTC_volatilityr], axis=1))
volatilityr_df.columns = ['BTC_rolling_volatility', 'ETH_rolling_volatility', 'LTC_rolling_volatility']

#joined columns into new dataframe and renamed
BTC_volatility = BTC_df['percent_volatility']
ETH_volatility = ETH_df['percent_volatility']
LTC_volatility = LTC_df['percent_volatility']
volatility_df = clean_new_data(pd.concat([BTC_volatility, ETH_volatility, LTC_volatility], axis=1))
volatility_df.columns = ['BTC_volatility', 'ETH_volatility', 'LTC_volatility']

#joined columns into new dataframe and renamed
BTC_high = BTC_df['high']
ETH_high = ETH_df['high']
LTC_high = LTC_df['high']
close_df = clean_new_data(pd.concat([BTC_high, ETH_high, LTC_high], axis=1))
close_df.columns = ['BTC_high', 'ETH_high', 'LTC_high']
ETH_LTC_close_df = clean_new_data(close_df.drop(columns='BTC_high'))

#joined columns into new dataframe and renamed
BTC_daily_change = BTC_df['daily_change']
ETH_daily_change = ETH_df['daily_change']
LTC_daily_change = LTC_df['daily_change']
daily_change_df = clean_new_data(pd.concat([BTC_daily_change, ETH_daily_change, LTC_daily_change], axis=1))
daily_change_df.columns = ['BTC_daily_change', 'ETH_daily_change', 'LTC_daily_change']


# Pull twitter data, then clean and create merged dataset

In [410]:
twitter_path = Path('../data/raw_data/raw_tweets_01_filter_polarity.csv')
twitter_df = pd.read_csv(twitter_path, infer_datetime_format=True, parse_dates=True)
datetime = twitter_df['time'].str.split(" ", n=1, expand = True)
twitter_df['date'] = datetime[0]
twitter_df.drop(columns='time')
twitter_df['time'] = datetime[1]
twitter_df.drop_duplicates(subset='text', inplace=True)
twitter_df.set_index('date', inplace=True)
#TODO: round all columns
df = pd.merge(twitter_df, volume_change_df, how='inner', left_index=True, right_index=True)
df2 = pd.merge(df, volatility_df, how='inner', left_index=True, right_index=True)
df3 = pd.merge(df2, close_df, how='inner', left_index=True, right_index=True)
df4 = pd.merge(df3, volatilityr_df, how='inner', left_index=True, right_index=True)
df5 = pd.merge(df4, rolling_volume_change_df, how='inner', left_index=True, right_index=True)
df6 = pd.merge(df5, volume_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df = pd.merge(df6, daily_change_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df['Popularity Rating'] = round(twitter_analysis_df['Popularity Rating'], 2)
twitter_analysis_df.sort_index(inplace=True)
twitter_analysis_df['category'].fillna('null', inplace=True)

#tweet_analysis_df['2020-03-30':'2021-19-01']
#df1.merge(df2, on='ID', how='left')
def export_df(df):
    return df.to_csv('../data/raw_data/raw_crypto_data.csv')
export_df(twitter_analysis_df)

# Here is some example data. We can add any tables and mess with the data from here. Anything we want to add on we should do through our crypto data frames and restart kernel

In [411]:
twitter_users = twitter_analysis_df['twitter_user'].unique()
crypto_category = twitter_analysis_df['category'].unique()
crypto_category
bitcoin_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('bit')]
ethereum_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('eth')]
twitter_users


array(['VitalikButerin', 'officialmcafee', 'joerogan', 'jack',
       'SatoshiLite', 'elonmusk'], dtype=object)

In [422]:
avg_df = twitter_analysis_df[twitter_analysis_df['category'] == 'bitcoin']


In [423]:
elon = twitter_users[5]
elon_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == elon]
elon_tweets1 = elon_tweets_df[elon_tweets_df['Popularity Rating'] != 0]
elon_tweets_rated = elon_tweets1[elon_tweets1['category'] == 'bitcoin']
elon_tweets_rated.drop(columns=['sentiment', 'time', 'tweet_id', 'tweet_source', 'quote_count', 'reply_count', 'retweet_count'], inplace=True)
def export_elon_data(df):
    return df.to_csv('../data/clean_data/clean_elon_data.csv')
export_elon_data(elon_tweets_rated)

C:\Users\miche\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
